# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'predict_maternal_risk'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-240923
Azure region: westus2
Subscription id: d4ad7261-832d-46b2-b093-22156001df5b
Resource group: aml-quickstarts-240923


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {'--C': uniform(0.05, 0.5),
    '--max_iter': choice(5, 10, 50, 100)
    } )

#TODO: Create your estimator and hyperdrive config
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target='notebook240923',
                      environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=param_sampling,
                                     policy=early_termination_policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=25,
                                     max_concurrent_runs=5)

In [6]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output = True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_1ea7ccc5-7b65-48ac-ada5-821f159e5df6
Web View: https://ml.azure.com/runs/HD_1ea7ccc5-7b65-48ac-ada5-821f159e5df6?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-240923/workspaces/quick-starts-ws-240923&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-09-01T15:03:27.728938][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space
[2023-09-01T15:03:28.6044614Z][SCHEDULER][INFO]Scheduling job, id='HD_1ea7ccc5-7b65-48ac-ada5-821f159e5df6_2' 
[2023-09-01T15:03:28.6022871Z][SCHEDULER][INFO]Scheduling job, id='HD_1ea7ccc5-7b65-48ac-ada5-821f159e5df6_0' 
[2023-09-01T15:03:28.6053454Z][SCHEDULER][INFO]Scheduling job, id='HD_1ea7ccc5-7b65-48ac-ada5-821f159e5df6_1' 
[2023-09-01T15:03:28.623335][GENERATOR][INFO]Successfully sampled '5' jobs, they will soon be submitted to the execution target.
[2023-09-01T15:03:28.9798283Z][SCHEDULER][INFO]Scheduling job, id='HD_1ea7ccc5-7b65-48ac-ada5-821f159e5d

{'runId': 'HD_1ea7ccc5-7b65-48ac-ada5-821f159e5df6',
 'target': 'notebook240923',
 'status': 'Completed',
 'startTimeUtc': '2023-09-01T15:03:26.76915Z',
 'endTimeUtc': '2023-09-01T15:22:52.373389Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'd862f684-56d6-4c67-bc22-b6b2ca19e3e9',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': 'infinite_space_size',
  'score': '1.0',
  'best_child_run_id': 'HD_1ea7ccc5-7b65-48ac-ada5-821f159e5df6_0',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_1ea7ccc5-7b65-48ac-ada5-821f159e5df6_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlCli

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [12]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print(best_run)
print('Accuracy:', best_run_metrics['Accuracy'])
print(f'Metrics: {best_run.get_metrics()}')

Run(Experiment: predict_maternal_risk,
Id: HD_1ea7ccc5-7b65-48ac-ada5-821f159e5df6_0,
Type: azureml.scriptrun,
Status: Completed)
Accuracy: 1.0
Metrics: {'Regularization Strength:': 0.37089225776052304, 'Max iterations:': 50, 'Accuracy': 1.0}


In [9]:
print(best_run.get_file_names())

['azureml-logs/20_image_build_log.txt', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


In [10]:
best_run.download_file(best_run.get_file_names()[0], output_file_path='./outputs/')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [11]:
best_run.register_model(model_name='best-run',
                                      model_path='outputs/model.joblib',
                                      description='Best model from Hyperdrive for maternal health risk')

Model(workspace=Workspace.create(name='quick-starts-ws-240923', subscription_id='d4ad7261-832d-46b2-b093-22156001df5b', resource_group='aml-quickstarts-240923'), name=best-run, id=best-run:1, version=1, tags={}, properties={})

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

